<a href="https://colab.research.google.com/github/anustupdas/document-sucject-classification/blob/main/Subject_Classification_Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Notebook for Training Document Subject Classifier with Custom Bert Model.

**In this notebook you can:**


1.   Access the Data preparation, Training, Evaluation and Inferencing code.
2.   Prepare Custom Dataset for training. [For now you can only prepare dataset for the given 56 classes present in the config file.]
3. Training a Bert-based classifier or Finetune a existing document subject classifier for now.
4. Run evaluation on any trainined document classifier model.
5. Call Inference of on any pre trained  document classifier model for any given text.



## **Get the code base**

The code base can be found


```
# https://github.com/anustupdas/document-sucject-classification.git
```




In [1]:
!git clone https://github.com/anustupdas/document-sucject-classification.git

Cloning into 'document-sucject-classification'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 88 (delta 43), reused 47 (delta 14), pack-reused 0
Unpacking objects: 100% (88/88), 826.45 KiB | 933.00 KiB/s, done.


In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


Install Requirmened dependencies

In [3]:
!pip install -r /content/document-sucject-classification/document-subject-classification-training/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 32.5 MB/s eta 0:00:00
     ━━━━━

In [4]:
cd /content/document-sucject-classification/document-subject-classification-training/document_subject_classification_training

/content/document-sucject-classification/document-subject-classification-training/document_subject_classification_training


# Data Preparation

**A prepared dataset is already available in the location**


tr_data_path = `/content/drive/MyDrive/medior-data-scientist-case-study/training_data_all_clean_chucked`
which is the output of `prepare_data.py` script.

*One can directly feed the datalocation for training by running*



`!python update_config.py --data_path tr_data_path`


### Alternative Create dataset from text files

One can either create the training dataset by running the below to create training testing dataframe from raw test files.


```
# prepare_data.py
```
This code reads the raw text files from the --data_path and does the following task:


1.   Cleans the text
2.   Breaks a text-file into multiple sub_text_file if the number of word count is higher than threshold. default token count now is 512.
3.  Divides the data into training validation and testing data
By default we split the data in 70, 18, 12 [train, val, test]. One can change the split percentage by passing a parameter         `--train_split .8`
4. creates respective dataframes [.csv] files for training testing and validation in the mention --output_data_path


In [ ]:
!python prepare_data.py --data_path "/content/drive/MyDrive/medior-data-scientist-case-study" --output_data_path "/content/Training_data"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                  Category  ...  Test
0                Economics  ...    11
1               Philosophy  ...     9
2              Criminology  ...     8
3                  Nursing  ...    11
4        Civil Engineering  ...     7
5                  Spanish  ...     4
6   Mechanical Engineering  ...     7
7               Management  ...    10
8             Food Science  ...     8
9             Architecture  ...    10
10       Religious Studies  ...    10
11             Agriculture  ...     7
12        Entrepreneurship  ...    10
13                   Logic  ...     6
14                 Physics  ...    10
15     Educational Science  ...     3
16                 History  ...     9
17             Probability  ...     9
18              Psychology  ...    11
19        

In [5]:
!python update_config.py --data_path "/content/drive/MyDrive/medior-data-scientist-case-study/training_data_all_clean_chucked"

In [6]:
cat config.json

{"tokenizer_type": "bert-base-multilingual-uncased", "model_type": "bert_classifier", "model_name": "bert-base-multilingual-uncased", "labels_class": {"Accounting": 0, "Aerospace Engineering": 1, "Agriculture": 2, "Algebra": 3, "Anthropology": 4, "Architecture": 5, "Astronomy": 6, "Biology": 7, "Calculus": 8, "Chemical Engineering": 9, "Chemistry": 10, "Civil Engineering": 11, "Communication Science": 12, "Computer Science": 13, "Criminology": 14, "Culinary Arts": 15, "Dentistry": 16, "Earth Science": 17, "Econometrics": 18, "Economics": 19, "Educational Science": 20, "Electrical Engineering": 21, "English": 22, "Entrepreneurship": 23, "Environmental Science": 24, "Finance": 25, "Food Science": 26, "French": 27, "Geography": 28, "Geological Science": 29, "Geometry": 30, "History": 31, "Industrial Design": 32, "Industrial Engineering": 33, "Law": 34, "Linguistics": 35, "Literature": 36, "Logic": 37, "Management": 38, "Mechanical Engineering": 39, "Medicine": 40, "Music": 41, "Nursing": 

## Training the Subject Classification bert multilingual model

In [ ]:
!python run.py --custom_data --model --train --epochs 4 --bs 10 --test_bs 8

/content/drive/MyDrive/medior-data-scientist-case-study/training_data_all_clean_chucked
File path to read:  /content/drive/MyDrive/medior-data-scientist-case-study/training_data_all_clean_chucked/train/clean_subject_classification_Train.csv
File path to read:  /content/drive/MyDrive/medior-data-scientist-case-study/training_data_all_clean_chucked/dev/clean_subject_classification_Val.csv
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS N

## Evaluation of the Model

In [ ]:
!python update_config.py --data_path "/content/Training_data"

In [ ]:
!python run.py --custom_data --load_from '/content/drive/MyDrive/Rcnn/best_model_14.t7'

/content/Training_data
File path to read:  /content/Training_data/test/token_main_data_subject_classification_summary_Test.csv
Model_Path:  /content/drive/MyDrive/Rcnn/best_model_14.t7
Here is model BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False

# Inferencing the Model

In [ ]:
!python run_inference.py --load_from "/content/drive/MyDrive/Rcnn/best_model_14.t7" --text "mon misconception: if critical criminologists state that crime real then they focus on harm (crime real but sufferings are) critical criminology: whole goal is to alleviate the we need to prevent harm, not proper subject is social harm crime catch the harm that happens to groups and we are not only individualizing on criminals but our legal framework also focuses on individual victims, ignoring group and societal crimes physical (example: illness, death from medical system not a crime but causes harm) financial (if we would criminalize being billionaire, criminologists would put all their forces to study but we glorify that emotional (tougher to see and respond pandemic restrictions,..) cultural (access to informational, intellectual, cultural resources (like access to they are par"

Model_Path:  /content/drive/MyDrive/Rcnn/best_model_14.t7
Criminology
